In [1]:
# import module
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.backend as backend
import sqlite3 as sql
import time
from sklearn.linear_model import LinearRegression
from lib.RNN_TVP import rescale_layer, linear_layer



In [2]:
# load data
data_raw = pd.read_csv('econ_data/three_variable.csv',header = None).dropna()
x_lag_1 = data_raw.loc[0:260,1:3].shift(1)
x_lag_2 = data_raw.loc[0:260,1:3].shift(2)
X = pd.concat([x_lag_1.loc[2:260,:],x_lag_2.loc[2:260,:]],axis=1)
X['intercept'] = np.ones([259,1])
y = data_raw.loc[2:260,1]

# transform data to tensor
x_train = tf.cast(X, tf.float32)
y_train = tf.cast(y, tf.float32)

In [3]:
# initial value by linea regression
reg = LinearRegression().fit(x_train, y_train)
beta_pre_sample = reg.coef_

# build the model
data_input = keras.Input(shape=(x_train.shape[-2], x_train.shape[-1]))
h_1 = keras.layers.SimpleRNN(16, return_sequences=True, name='RNN_1')(data_input)
h_2 = keras.layers.LSTM(16, return_sequences=True, name='RNN_2')(h_1)
beta_init = tf.constant_initializer(beta_pre_sample)
h_fix = keras.layers.Dense(1, name='fix', kernel_initializer=beta_init,
                           kernel_regularizer=keras.regularizers.l1(0.1))(data_input)
tvp = rescale_layer(16, x_train, x_train.shape[-1], name='rescale', regularizers = keras.regularizers.l1(0.1))(h_2)
linear = linear_layer(3, x_train, name='linear')(tvp, h_fix)
model = keras.Model(inputs=data_input, outputs=linear)
model.compile(optimizer="Adam", loss="mse", metrics=["mae"])

# model fit
model.fit(x=x_train[tf.newaxis, :, :], y=y_train[tf.newaxis, :, tf.newaxis], epochs=2000)

Epoch 1/2000


2022-06-09 14:32:15.829243: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


1/1 [==============================] - 1s 1s/step - loss: 21.2101 - mae: 2.8380
Epoch 2/2000
1/1 [==============================] - 0s 34ms/step - loss: 20.4056 - mae: 2.8333
Epoch 3/2000
1/1 [==============================] - 0s 35ms/step - loss: 19.6996 - mae: 2.8243
Epoch 4/2000
1/1 [==============================] - 0s 29ms/step - loss: 19.0598 - mae: 2.8137
Epoch 5/2000
1/1 [==============================] - 0s 34ms/step - loss: 18.4834 - mae: 2.8008
Epoch 6/2000
1/1 [==============================] - 0s 34ms/step - loss: 17.9671 - mae: 2.7865
Epoch 7/2000
1/1 [==============================] - 0s 34ms/step - loss: 17.5171 - mae: 2.7733
Epoch 8/2000
1/1 [==============================] - 0s 32ms/step - loss: 17.1264 - mae: 2.7605
Epoch 9/2000
1/1 [==============================] - 0s 32ms/step - loss: 16.8029 - mae: 2.7484
Epoch 10/2000
1/1 [==============================] - 0s 34ms/step - loss: 16.5542 - mae: 2.7376
Epoch 11/2000
1/1 [==============================] - 0s 34ms/ste

In [58]:
# save output

# coefficients
layer = model.get_layer('rescale')
extractor = keras.Model(inputs=model.inputs,
                        outputs=layer.output)
features = np.array(extractor(x_train[tf.newaxis, :, :]))[0, :, :]
fix_layer = model.get_layer('fix')
beta_fix = np.array(fix_layer.get_weights())[0]
beta_final = features + np.transpose(beta_fix)

# fitted y
layer_1 = model.get_layer('linear')
extractor = keras.Model(inputs=model.inputs,
                        outputs=layer_1.output)
y_hat = np.array(extractor(x_train[tf.newaxis, :, :]))[0, :]

# save the result into database
save_table = pd.DataFrame(beta_final)
save_table.insert(0, 'fitted_y', y_hat)
save_table.insert(0, 'y_true', y_train)
save_table.to_csv('result/in_sample_fit_RNN.csv')

/var/folders/vl/6jzn2pl97hl56zn79gt080440000gn/T/ipykernel_56253/842336546.py:8: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  beta_fix = np.array(fix_layer.get_weights())[0]
